In [2]:
interp.load.ivy(
  coursier.Dependency(
    module = coursier.Module("org.platanios", "tensorflow_2.12"),
    version = "0.4.1",
    // replace with linux-gpu-x86_64 on linux with nvidia gpu or with darwin-cpu-x86_64 on macOS 
    attributes = coursier.Attributes("", "darwin-cpu-x86_64")
  )
)
interp.load.ivy("org.platanios" %% "tensorflow-data" % "0.4.1")

In [3]:
import org.platanios.tensorflow.api._
import org.platanios.tensorflow.api.learn._
import org.platanios.tensorflow.api.learn.layers._
import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator
import org.platanios.tensorflow.data.image.MNISTLoader
import org.platanios.tensorflow.api.core.client.FeedMap
import org.tensorflow.framework.GraphDef

import org.platanios.tensorflow.api.ops.Files
import org.platanios.tensorflow.api.ops.Image

import org.platanios.tensorflow.api._

import org.platanios.tensorflow.api.learn._

import org.platanios.tensorflow.api.learn.layers._

import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator

import org.platanios.tensorflow.data.image.MNISTLoader

import org.platanios.tensorflow.api.core.client.FeedMap

import org.tensorflow.framework.GraphDef


import org.platanios.tensorflow.api.ops.Files

import org.platanios.tensorflow.api.ops.Image

In [4]:
import java.net.URL
import sys.process._
import java.io.{BufferedInputStream, File, FileInputStream}

import java.net.URL

import sys.process._

import java.io.{BufferedInputStream, File, FileInputStream}

## Prepare the model to be served

### We download the published Tensorflow model from a public URL

See https://github.com/tensorflow/models for reference on available models from research community.

In [5]:
val cacheDir = sys.env("HOME") + "/data/models/tmp"
val modelName = "ssd_mobilenet_v2_coco_2018_03_29"
val archiveFilename = s"${modelName}.tar.gz"

val modelURL = s"http://download.tensorflow.org/models/object_detection/${archiveFilename}"

cacheDir: String = "/Users/xavier/data/models/tmp"
modelName: String = "ssd_mobilenet_v2_coco_2018_03_29"
archiveFilename: String = "ssd_mobilenet_v2_coco_2018_03_29.tar.gz"
modelURL: String = "http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz"

In [ ]:
new URL(modelURL) #> new File(s"${cacheDir}/${archiveFilename}") !!

### Extraction of the model archive


In [ ]:
s"tar -xzf ${cacheDir}/${archiveFilename} -C ${cacheDir}" !!

In [6]:
s"ls ${cacheDir}/${modelName}"!

checkpoint
frozen_inference_graph.pb
model.ckpt.data-00000-of-00001
model.ckpt.index
model.ckpt.meta
mscoco_label_map.pbtxt
pipeline.config
saved_model


res5: Int = 0

### Load the model in a Tensorflow Session

In the extracted directory, named after the `modelName`, we are interested in loading the `frozen_inference_graph.pb`.

In [7]:
val modelGraphPath = s"${cacheDir}/${modelName}/frozen_inference_graph.pb"

modelGraphPath: String = "/Users/xavier/data/models/tmp/ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb"

In [8]:
lazy val graphDef = GraphDef.parseFrom(
    new BufferedInputStream(new FileInputStream(new File(modelGraphPath))))
val graph = Graph.fromGraphDef(graphDef)
val session = Session(graph)

graphDef: GraphDef = <lazy>
graph: Graph = org.platanios.tensorflow.api.core.Graph@b436c0d
session: Session = org.platanios.tensorflow.api.core.client.Session@47766d4c

### Add nodes to access the model signature (input and responses)

`image_tensor` is the tensor representing the input images, with 3-channels colors

In [9]:
val imagePlaceholder = graph.getOutputByName("image_tensor:0").toUByte
imagePlaceholder.shape

imagePlaceholder: Output[UByte] = Output(image_tensor, 0)
res8_1: Shape = [?, ?, ?, 3]

`num_detections` is the number of detected objects

`detection_boxes` are the corrdinates of detected objects

`detection_scores` are a probability-like score for each object

`detection_classes` are the class label for each detection

In [10]:
val detectionBoxes = graph.getOutputByName("detection_boxes:0")
val detectionScores = graph.getOutputByName("detection_scores:0")
val detectionClasses = graph.getOutputByName("detection_classes:0")
val numDetections = graph.getOutputByName("num_detections:0")

detectionBoxes: Output[Any] = Output(detection_boxes, 0)
detectionScores: Output[Any] = Output(detection_scores, 0)
detectionClasses: Output[Any] = Output(detection_classes, 0)
numDetections: Output[Any] = Output(num_detections, 0)

### Nodes to feed the graph with an image file path

A Placeholder to provide a filepathe is the input, and the file is opened and decoded as an image:

In [12]:
val (imgTensor, fileNamePlaceholder) = tf.createWith(graph = graph) {
    val fileNamePlaceholder = tf.placeholder[String]()
    val fileTensor = Files.readFile(fileNamePlaceholder)
    val imgTensor = Image.decodePng(fileTensor, 3)
    (imgTensor, fileNamePlaceholder)
  }

imgTensor: Output[UByte] = Output(DecodePng_1, 0)
fileNamePlaceholder: Output[String] = Output(Placeholder_1, 0)

## Define the Service function


In [13]:
def detectObjects(file: File) = {
    
    // Feed the image file to get the Images Tensor
    val fileNameTensor = Tensor.fill(Shape())(file.getAbsolutePath())
    val feedImg = FeedMap(fileNamePlaceholder, fileNameTensor)
    val imageOuts: Tensor[UByte] =
      session.run(fetches = imgTensor, feeds = feedImg)

    // Retain image sizes to format output later
    val width = imageOuts.shape(1)
    val height = imageOuts.shape(0)

    // Feed with Images to compute detections:
    val feeds = FeedMap(imagePlaceholder, imageOuts.slice(NewAxis, ---))
    val Seq(boxes, scores, classes, num) =
      session.run(
        fetches =
          Seq(detectionBoxes, detectionScores, detectionClasses, numDetections),
        feeds = feeds)
    
  val labelList =
      for {
        i <- 0 until num(0).scalar.asInstanceOf[Float].toInt
        labelId = classes(0, i).toFloat.scalar.toInt
        //label = labelMap.getOrElse(labelId, "unknown")
        //if setOfClasses.isEmpty || setOfClasses.contains(label)

        box = boxes(0, i).toFloat.entriesIterator.toSeq
        x1 = (box(1) * width).toInt
        y1 = (box(0) * height).toInt
        x2 = (box(3) * width).toInt
        y2 = (box(2) * height).toInt
        labelBox = (x1, y1, x2 - x1 + 1, y2 - y1 + 1)
        score = scores(0, i).toFloat.scalar
      } yield (labelId, score, labelBox)
    labelList.toSeq
}

defined function detectObjects

In [15]:
detectObjects(new File(sys.env("HOME") + "/Downloads/baywatch.jpg"))

res14: collection.immutable.Seq[(Int, Float, (Int, Int, Int, Int))] = Vector(
  (1, 0.9714311F, (141, 38, 368, 722)),
  (1, 0.96262753F, (554, 101, 328, 651))
)

## Transform Labels ids in labels

We need a mapping between class id and class name for detections. This is provided as protobuf data and message definitions:

See https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt


In [16]:
val mapFilename = "mscoco_label_map.pbtxt"
val mapUrl = s"https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/${mapFilename}"


mapFilename: String = "mscoco_label_map.pbtxt"
mapUrl: String = "https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt"

In [ ]:
new URL(mapUrl) #> new File(s"${cacheDir}/${modelName}/${mapFilename}") !

In [17]:
import $ivy.`default:models_proto_2.12:0.1.0-SNAPSHOT`
import object_detection.protos.string_int_label_map.{StringIntLabelMap,StringIntLabelMapItem}

import $ivy.$                                         

import object_detection.protos.string_int_label_map.{StringIntLabelMap,StringIntLabelMapItem}

In [18]:
import $ivy.`io.circe:circe-core_2.12:0.10.1`
import $ivy.`io.circe:circe-generic_2.12:0.10.1`
import $ivy.`io.circe:circe-parser_2.12:0.10.1`
import _root_.io.circe.{Decoder, Encoder}

case class Detection(`class`: String, score: Float, box: (Int, Int, Int, Int))

object Detection {
  import _root_.io.circe.generic.semiauto._

  implicit lazy val encoder: Encoder[Detection] = deriveEncoder[Detection]
  implicit lazy val decoder: Decoder[Detection] = deriveDecoder[Detection]
}

import $ivy.$                                

import $ivy.$                                   

import $ivy.$                                  

import _root_.io.circe.{Decoder, Encoder}


defined class Detection
defined object Detection

In [19]:
val labelMap: scala.collection.Map[Int, String] = {
    val pbText = scala.io.Source.fromFile(s"${cacheDir}/${modelName}/${mapFilename}").mkString
    val stringIntLabelMap = StringIntLabelMap.fromAscii(pbText)
    stringIntLabelMap.item.collect {
      case StringIntLabelMapItem(_, Some(id), Some(displayName)) =>
        id -> displayName
    }.toMap
  }

labelMap: collection.Map[Int, String] = Map(
  88 -> "teddy bear",
  5 -> "airplane",
  10 -> "traffic light",
  56 -> "broccoli",
  42 -> "surfboard",
  24 -> "zebra",
  37 -> "sports ball",
  25 -> "giraffe",
  52 -> "banana",
  14 -> "parking meter",
  20 -> "sheep",
  46 -> "wine glass",
  57 -> "carrot",
  78 -> "microwave",
  84 -> "book",
  61 -> "cake",
  89 -> "hair drier",
  1 -> "person",
  74 -> "mouse",
  6 -> "bus",
  60 -> "donut",
  85 -> "clock",
  28 -> "umbrella",
  38 -> "kite",
  70 -> "toilet",
  21 -> "cow",
  33 -> "suitcase",
  65 -> "bed",
  9 -> "boat",
  53 -> "apple",
  77 -> "cell phone",
  13 -> "stop sign",
  41 -> "skateboard",
  73 -> "laptop",
  2 -> "bicycle",
  32 -> "tie",
  34 -> "frisbee",
  64 -> "potted plant",
...

In [20]:
def labelize(detections: Seq[(Int, Float, (Int, Int, Int, Int))]) = detections.map { 
    d => d match {
        case (id, score, box) if (labelMap.contains(id)) => Detection(labelMap(id), score, box)
        case (id, score, box) => Detection("unknown", score, box)
    }
}

defined function labelize

In [21]:
labelize( detectObjects(new File(sys.env("HOME") + "/Downloads/baywatch.jpg")) )

res20: Seq[Detection] = Vector(
  Detection("person", 0.9714311F, (141, 38, 368, 722)),
  Detection("person", 0.96262753F, (554, 101, 328, 651))
)

## Run an akka-http service

In [22]:
import $ivy.`com.typesafe.akka:akka-http_2.12:10.1.5`
import $ivy.`com.typesafe.akka:akka-actor_2.12:2.5.18`
import $ivy.`com.typesafe.akka:akka-stream_2.12:2.5.18`
import $ivy.`de.heikoseeberger:akka-http-circe_2.12:1.21.0`

import akka.actor.ActorSystem
import akka.stream.{IOResult, Materializer}
import akka.stream.scaladsl.{FileIO, Source}
import akka.http.scaladsl.Http
import akka.http.scaladsl.Http.ServerBinding
import akka.http.scaladsl.server.Directives._
import akka.http.scaladsl.server.Route
import akka.stream.ActorMaterializer
import akka.util.ByteString

import scala.util.{Failure, Success}
import java.nio.file.{Files, Paths}

import $ivy.$                                        

import $ivy.$                                         

import $ivy.$                                          

import $ivy.$                                              


import akka.actor.ActorSystem

import akka.stream.{IOResult, Materializer}

import akka.stream.scaladsl.{FileIO, Source}

import akka.http.scaladsl.Http

import akka.http.scaladsl.Http.ServerBinding

import akka.http.scaladsl.server.Directives._

import akka.http.scaladsl.server.Route

import akka.stream.ActorMaterializer

import akka.util.ByteString


import scala.util.{Failure, Success}

import java.nio.file.{Files, Paths}

In [23]:
implicit val system = ActorSystem("main-system")
implicit val materializer = ActorMaterializer()
implicit val executionContext = system.dispatcher

import de.heikoseeberger.akkahttpcirce.FailFastCirceSupport._

val route =
  path("detect") {
      post {
          fileUpload("img") {
              case (metadata, byteSource) =>
              val file = File.createTempFile("image", ".png")
              val fileSaveFut = byteSource.runWith(FileIO.toPath(Paths.get(file.getAbsolutePath)))
              onComplete(fileSaveFut) {
                  case Success(s) => 
                      val detections = labelize( detectObjects(file))
                      complete(detections)
                  case Failure(s) => complete(s.getMessage)
              }
          }
      }
  }



system: ActorSystem = akka://main-system
materializer: ActorMaterializer = PhasedFusingActorMaterializer(
  akka://main-system,
  ActorMaterializerSettings(4,16,,<function1>,StreamSubscriptionTimeoutSettings(CancelTermination,5000 milliseconds),false,1000,1000,false,true,IoSettings(16384)),
  akka.dispatch.Dispatchers@40732850,
  Actor[akka://main-system/system/StreamSupervisor-0#1015632836],
  false,
  akka.stream.impl.SeqActorNameImpl@1be9bf24
)
executionContext: concurrent.ExecutionContextExecutor = Dispatcher[akka.actor.default-dispatcher]
import de.heikoseeberger.akkahttpcirce.FailFastCirceSupport._


route: akka.http.scaladsl.server.RequestContext => concurrent.Future[akka.http.scaladsl.server.RouteResult] = akka.http.scaladsl.server.directives.BasicDirectives$$Lambda$4227/414237822@7cd1986b

In [24]:
val bindingFuture = Http().bindAndHandle(route, "localhost", 8080)

bindingFuture: concurrent.Future[ServerBinding] = Success(ServerBinding(/127.0.0.1:8080))

In [ ]:
bindingFuture
      .flatMap(_.unbind()) // trigger unbinding from the port
      .onComplete(_ => system.terminate()) // and shutdown when done
